In [1]:
import numpy as np
import pandas as pd
import warnings

In [2]:
sales_train = pd.read_csv('./data/sales_train.csv')
shops = pd.read_csv('./data/shops.csv')
items = pd.read_csv('./data/items.csv')
item_categories = pd.read_csv('./data/item_categories.csv')
test = pd.read_csv('./data/test.csv')

In [3]:
print(sales_train.shape)
print(shops.shape)
print(items.shape)
print(item_categories.shape)
print(test.shape)

(2935849, 6)
(60, 2)
(22170, 3)
(84, 2)
(214200, 3)


In [4]:
sales_train.head()  # 월별 판매량만 구하려면 data 피처를 제거
                    # Target 값 = data_block_num 피처의 값이 같은 데이터들의 item_cnt_data값의 합임.
                    # 훈련 데이터 중 가장 최근인 2015년 10월 판매 내역을 검증 데이터로 사용

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.00,1.0
1,03.01.2013,0,25,2552,899.00,1.0
2,05.01.2013,0,25,2552,899.00,-1.0
3,06.01.2013,0,25,2554,1709.05,1.0
4,15.01.2013,0,25,2555,1099.00,1.0


In [5]:
sales_train.info()  # 메모리 절감 전략이 필요함.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2935849 entries, 0 to 2935848
Data columns (total 6 columns):
 #   Column          Dtype  
---  ------          -----  
 0   date            object 
 1   date_block_num  int64  
 2   shop_id         int64  
 3   item_id         int64  
 4   item_price      float64
 5   item_cnt_day    float64
dtypes: float64(2), int64(3), object(1)
memory usage: 134.4+ MB


In [6]:
shops.head()  # 상점 데이터 shop_name 의 첫 단어는 도시 이름임.
              # train_sale의 shop_id를 기준으로 sales_train과 shops 데이터 병합 필요

,shop_name,shop_id
0,"!Якутск Орджоникидзе, 56 фран",0
1,"!Якутск ТЦ ""Центральный"" фран",1
2,"Адыгея ТЦ ""Мега""",2
3,"Балашиха ТРК ""Октябрь-Киномир""",3
4,"Волжский ТЦ ""Волга Молл""",4


In [7]:
shops.info()   # 상점 데이터 60개로 결측 데이터가 없음.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60 entries, 0 to 59
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   shop_name  60 non-null     object
 1   shop_id    60 non-null     int64 
dtypes: int64(1), object(1)
memory usage: 1.1+ KB


In [8]:
items.head()  # item 데이터에서 item_category_id 제거
              # item_id를 기준으로 sales_train과 items 병합

,item_name,item_id,item_category_id
0,! ВО ВЛАСТИ НАВАЖДЕНИЯ (ПЛАСТ.) D,0,40
1,!ABBYY FineReader 12 Professional Edition Full...,1,76
2,***В ЛУЧАХ СЛАВЫ (UNV) D,2,40
3,***ГОЛУБАЯ ВОЛНА (Univ) D,3,40
4,***КОРОБКА (СТЕКЛО) D,4,40


In [9]:
items.info()  # item 데이터는 결측 데이터가 없음

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22170 entries, 0 to 22169
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   item_name         22170 non-null  object
 1   item_id           22170 non-null  int64 
 2   item_category_id  22170 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 519.7+ KB


In [10]:
item_categories.head()   # item_category_id를 기준으로 sales_train과 item_categories 데이터 병합 필요
                         # item_category_name의 첫 단어는 대분류로 분류하여야 함

,item_category_name,item_category_id
0,PC - Гарнитуры/Наушники,0
1,Аксессуары - PS2,1
2,Аксессуары - PS3,2
3,Аксессуары - PS4,3
4,Аксессуары - PSP,4


In [11]:
item_categories.info()  # 모든 데이터의 결측값 없음

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 84 entries, 0 to 83
Data columns (total 2 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   item_category_name  84 non-null     object
 1   item_category_id    84 non-null     int64 
dtypes: int64(1), object(1)
memory usage: 1.4+ KB


In [12]:
test.head()

,ID,shop_id,item_id
0,0,5,5037
1,1,5,5320
2,2,5,5233
3,3,5,5232
4,4,5,5268


# 01. 피처명 한글화

## 01. sales_train

In [13]:
sales_train = sales_train.rename(columns={'date': '날짜',
                                          'date_block_num': '월ID',
                                          'shop_id': '상점ID',
                                          'item_id': '상품ID',
                                          'item_price': '판매가',
                                          'item_cnt_day': '판매량'})

In [14]:
sales_train.head()

,날짜,월ID,상점ID,상품ID,판매가,판매량
0,02.01.2013,0,59,22154,999.00,1.0
1,03.01.2013,0,25,2552,899.00,1.0
2,05.01.2013,0,25,2552,899.00,-1.0
3,06.01.2013,0,25,2554,1709.05,1.0
4,15.01.2013,0,25,2555,1099.00,1.0


## 02. shop

In [15]:
shops = shops.rename(columns={'shop_name': '상점명',
                              'shop_id': '상점ID'})

In [16]:
shops.head()

,상점명,상점ID
0,"!Якутск Орджоникидзе, 56 фран",0
1,"!Якутск ТЦ ""Центральный"" фран",1
2,"Адыгея ТЦ ""Мега""",2
3,"Балашиха ТРК ""Октябрь-Киномир""",3
4,"Волжский ТЦ ""Волга Молл""",4


## 03. items

In [17]:
items = items.rename(columns={'item_name': '상품명',
                              'item_id': '상품ID',
                              'item_category_id': '상품분류ID'})

In [18]:
items.head()

,상품명,상품ID,상품분류ID
0,! ВО ВЛАСТИ НАВАЖДЕНИЯ (ПЛАСТ.) D,0,40
1,!ABBYY FineReader 12 Professional Edition Full...,1,76
2,***В ЛУЧАХ СЛАВЫ (UNV) D,2,40
3,***ГОЛУБАЯ ВОЛНА (Univ) D,3,40
4,***КОРОБКА (СТЕКЛО) D,4,40


## 04. item_categories

In [19]:
item_categories = item_categories.rename(columns=
                                         {'item_category_name': '상품분류명',
                                          'item_category_id': '상품분류ID'})

In [20]:
item_categories.head()

,상품분류명,상품분류ID
0,PC - Гарнитуры/Наушники,0
1,Аксессуары - PS2,1
2,Аксессуары - PS3,2
3,Аксессуары - PS4,3
4,Аксессуары - PSP,4


## 05. test

In [21]:
test = test.rename(columns={'shop_id': '상점ID',
                            'item_id': '상품ID'})

In [22]:
test.head()

,ID,상점ID,상품ID
0,0,5,5037
1,1,5,5320
2,2,5,5233
3,3,5,5232
4,4,5,5268


# 02. 다운캐스팅이란?

* 다운캐스팅(downcasting)이란 더 작은 데이터 타입으로 변환하는 작업
* 금붕어는 금붕어용 어항에서. 돌고래용 수족관을 마련할 필요는 없다!
* 메모리 낭비를 막고, 훈련 속도를 빠르게 하려면 모든 데이터를 최대 타입으로 할당할 필요는 없음(분석 정리 5)

In [24]:
def downcast(df, verbose=True):
    start_mem=df.memory_usage().sum()/1024**2  # ( data fram을 가지고 있는 array 공간 차지 절감을 위해 )
    for col in df.columns:
        dtype_name = df[col].dtype.name
        if dtype_name == 'object':
            pass
        elif dtype_name == 'bool':
            df[col] = df[col].astype('int8')
        elif dtype_name.startswith('int') or (df[col].round() == df[col]).all():
            df[col] = pd.to_numeric(df[col], downcast='integer')
        else:
            df[col] = pd.to_numeric(df[col], downcast='float')
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose:
        print('{:.1f}% 압축됨'.format(100 * (start_mem - end_mem) / start_mem))
    return df

In [25]:
all_df = [sales_train, shops, items, item_categories, test]
for df in all_df:
    df = downcast(df)

62.5% 압축됨
38.6% 압축됨
54.2% 압축됨
39.9% 압축됨
70.8% 압축됨


In [26]:
all_df

[                 날짜  월ID  상점ID   상품ID          판매가  판매량
 0        02.01.2013    0    59  22154   999.000000    1
 1        03.01.2013    0    25   2552   899.000000    1
 2        05.01.2013    0    25   2552   899.000000   -1
 3        06.01.2013    0    25   2554  1709.050049    1
 4        15.01.2013    0    25   2555  1099.000000    1
 ...             ...  ...   ...    ...          ...  ...
 2935844  10.10.2015   33    25   7409   299.000000    1
 2935845  09.10.2015   33    25   7460   299.000000    1
 2935846  14.10.2015   33    25   7459   349.000000    1
 2935847  22.10.2015   33    25   7440   299.000000    1
 2935848  03.10.2015   33    25   7460   299.000000    1
 
 [2935849 rows x 6 columns],
                                                 상점명  상점ID
 0                     !Якутск Орджоникидзе, 56 фран     0
 1                     !Якутск ТЦ "Центральный" фран     1
 2                                  Адыгея ТЦ "Мега"     2
 3                    Балашиха ТРК "Октябрь-Кином

In [27]:
from itertools import product

In [28]:
train = []

In [29]:
for i in sales_train['월ID'].unique():
    all_shop = sales_train.loc[sales_train['월ID']==i, '상점ID'].unique()
    all_item = sales_train.loc[sales_train['월ID']==i, '상품ID'].unique()
    train.append(np.array(list(product([i], all_shop, all_item))))

idx_features = ['월ID', '상점ID', '상품ID'] # 기준 피처

# 리스트 타입인 train을 DataFrame 타입으로 변환
train = pd.DataFrame(np.vstack(train), columns=idx_features)

In [30]:
train

,월ID,상점ID,상품ID
0,0,59,22154
1,0,59,2552
2,0,59,2554
3,0,59,2555
4,0,59,2564
...,...,...,...
10913845,33,21,7635
10913846,33,21,7638
10913847,33,21,7640
10913848,33,21,7632


# 03. 타깃 값인 각 상점의 상품별 월간 판매량 추가

* sales_train에 있는 일별 판매량 피처를 그룹화해서
  우리가 원하는 타깃값인 각 상점의 상품별 '월간' 판매량 생성
  
* 앞서 피처 조합을 만들 때 생성한
  idx_features = ['월ID', '상점ID', '상품ID']를 기준으로 그룹화

In [31]:
# idx_features를 기준으로 그룹화해 판매량 합 구하기
group = sales_train.groupby(idx_features).agg({'판매량': 'sum'})

# 인덱스 재설정
group = group.reset_index()

# '월간 판매량'을 구했으니 피처명 '판매량'도 '월간 판매량'으로 변경
# 피처명을 '판매량'에서 '월간 판매량'으로 변경
group = group.rename(columns={'판매량': '월간 판매량'}) 
group

,월ID,상점ID,상품ID,월간 판매량
0,0,0,32,6
1,0,0,33,3
2,0,0,35,1
3,0,0,43,1
4,0,0,51,2
...,...,...,...,...
1609119,33,59,22087,6
1609120,33,59,22088,2
1609121,33,59,22091,1
1609122,33,59,22100,1


# 04. 타깃값(월간 판매량) 추가

* train과 group을 병합
  - train은 월ID, 상점ID, 상품ID의 조합
  - train에 group을 병합하면 월ID, 상점ID, 상품ID, 월간 판매량 조합을 나타냄

In [32]:
# train과 group 병합하기
train = train.merge(group, on=idx_features, how='left')
train

,월ID,상점ID,상품ID,월간 판매량
0,0,59,22154,1.0
1,0,59,2552,NaN
2,0,59,2554,NaN
3,0,59,2555,NaN
4,0,59,2564,NaN
...,...,...,...,...
10913845,33,21,7635,NaN
10913846,33,21,7638,NaN
10913847,33,21,7640,NaN
10913848,33,21,7632,NaN


* 우리가 원하는 타깃값을 잘 만듦
* 그런데 타깃값에 결측값이 많음 (결측값은 판매량 정보가 없다는 뜻이므로 추후 0으로 대체)
* 또한, train을 만드는 과정에서 sales_train에 있던 date(날짜) 피처가 사라짐
  필요 없는 date 피처를 명시적으로 제거한 게 아니라,
* 병합하는 과정에서 제외하여 같은 효과를 얻음(분석 정리 7)

# 05. 가비지 컬렉션

* group 데이터는 더 이상 필요 없으니 가비지 컬렉션 garbage collection
  메모리 절약 차원에서 가비지 컬렉션 수행(분석 정리 5)

In [33]:
group

,월ID,상점ID,상품ID,월간 판매량
0,0,0,32,6
1,0,0,33,3
2,0,0,35,1
3,0,0,43,1
4,0,0,51,2
...,...,...,...,...
1609119,33,59,22087,6
1609120,33,59,22088,2
1609121,33,59,22091,1
1609122,33,59,22100,1


In [34]:
import gc # 가비지 컬렉터 불러오기
del group # 더는 사용하지 않는 변수 지정
gc.collect(); # 가비지 컬렉션 수행

In [35]:
group

NameError: name 'group' is not defined

- 한정된 메모리를 효율적으로 사용하려면 틈틈이 가비지 컬렉션해주는 게 좋음

# 06. 테스트 데이터 이어붙이기

* 지금까지 한 작업
* 월ID, 상점ID, 상품ID 조합으로 train을 만들고, 여기에 월간 판매량(타깃값)을 추가함
* 테스트 데이터 이어붙이기
  - 뒤이어 shops, items, item_categories 데이터를 병합할 텐데, 이때 테스트 데이터에도
    한 번에 병합하는 게 좋기 때문 우선, 월ID 피처 추가
    (2015년 10월의 월ID가 33이므로 2015년 11월의 월ID는 34)

In [36]:
test['월ID'] = 34

# train과 test 이어붙이기
all_data = pd.concat([train, test.drop('ID', axis=1)],
                     ignore_index=True, # 기존 인덱스 무시(0부터 새로 시작)
                     keys=idx_features) # 이어붙이는 기준이 되는 피처

In [37]:
all_data

,월ID,상점ID,상품ID,월간 판매량
0,0,59,22154,1.0
1,0,59,2552,NaN
2,0,59,2554,NaN
3,0,59,2555,NaN
4,0,59,2564,NaN
...,...,...,...,...
11128045,34,45,18454,NaN
11128046,34,45,16188,NaN
11128047,34,45,15757,NaN
11128048,34,45,19648,NaN


In [ ]:
# ( 0월은 2013년 1월이며, 0-33까지가 기존의 데이터임. 34는 2015년 11월 )

- 결측값은 0으로 대체
  - train과 group을 병합할 때 결측값이 많았기 때문에

In [38]:
# 결측값을 0으로 대체
all_data = all_data.fillna(0)
all_data

,월ID,상점ID,상품ID,월간 판매량
0,0,59,22154,1.0
1,0,59,2552,0.0
2,0,59,2554,0.0
3,0,59,2555,0.0
4,0,59,2564,0.0
...,...,...,...,...
11128045,34,45,18454,0.0
11128046,34,45,16188,0.0
11128047,34,45,15757,0.0
11128048,34,45,19648,0.0


# 07. 나머지 데이터 병합(최종 데이터 생성)

- 나머지 데이터를 all_data에 병합(분석 정리 4)
- 추가로, 데이터 다운캐스팅과 가비지 컬렉션 수행(분석 정리 5)

In [39]:
# 나머지 데이터 병합
all_data = all_data.merge(shops, on='상점ID', how='left')
all_data = all_data.merge(items, on='상품ID', how='left')
all_data = all_data.merge(item_categories, on='상품분류ID', how='left')

# 데이터 다운캐스팅
all_data = downcast(all_data)

# 가비지 컬렉션
del shops, items, item_categories
gc.collect();

26.4% 압축됨


# 08. 병합한 데이터 확인

In [40]:
all_data.head()

,월ID,상점ID,상품ID,월간 판매량,상점명,상품명,상품분류ID,상품분류명
0,0,59,22154,1,"Ярославль ТЦ ""Альтаир""",ЯВЛЕНИЕ 2012 (BD),37,Кино - Blu-Ray
1,0,59,2552,0,"Ярославль ТЦ ""Альтаир""",DEEP PURPLE The House Of Blue Light LP,58,Музыка - Винил
2,0,59,2554,0,"Ярославль ТЦ ""Альтаир""",DEEP PURPLE Who Do You Think We Are LP,58,Музыка - Винил
3,0,59,2555,0,"Ярославль ТЦ ""Альтаир""",DEEP PURPLE 30 Very Best Of 2CD (Фирм.),56,Музыка - CD фирменного производства
4,0,59,2564,0,"Ярославль ТЦ ""Альтаир""",DEEP PURPLE Perihelion: Live In Concert DVD (К...,59,Музыка - Музыкальное видео


* 상점명, 상품명, 상품분류명 피처 제거

* 문자 데이터이기도 하거니와, 상점ID, 상품ID, 상품분류ID와 일대일로 매칭돼서 제거해도 됨(분석 정리8)

In [41]:
all_data =all_data.drop(['상점명', '상품명', '상품분류명'], axis=1)

In [42]:
all_data.head()   # ( 순수하게 계산할 수 있는 Data만 남게 됨 )

,월ID,상점ID,상품ID,월간 판매량,상품분류ID
0,0,59,22154,1,37
1,0,59,2552,0,58
2,0,59,2554,0,58
3,0,59,2555,0,56
4,0,59,2564,0,59


# 09. 마무리
## 훈련, 검증, 테스트 데이터로 나누기

In [43]:
# 훈련 데이터 (피처)
X_train = all_data[all_data['월ID'] < 33]
X_train = X_train.drop(['월간 판매량'], axis=1)

# 검증 데이터 (피처)
X_valid = all_data[all_data['월ID'] == 33]
X_valid = X_valid.drop(['월간 판매량'], axis=1)

# 테스트 데이터 (피처)
X_test = all_data[all_data['월ID'] == 34]
X_test = X_test.drop(['월간 판매량'], axis=1)

# 훈련 데이터 (타깃값)
y_train = all_data[all_data['월ID'] < 33]['월간 판매량']
y_train = y_train.clip(0, 20) # 타깃값을 0 ~ 20로 제한

# 검증 데이터 (타깃값)
y_valid = all_data[all_data['월ID'] == 33]['월간 판매량']
y_valid = y_valid.clip(0, 20)

* LightGBM을 활용해 베이스라인 모델 훈련

In [ ]:
# ( 34번 데이터는 예측을 위해 test data )

In [44]:
import lightgbm as lgb

# LightGBM용 하이퍼파라미터
params = {'metric': 'rmse', # 평가지표 = rmse
          'num_leaves': 255, 'learning_rate': 0.01,
          'force_col_wise': True, 'random_state': 10}

cat_features = ['상점ID', '상품분류ID'] # 범주형 피처 설정

# LightGBM용 훈련 및 검증 데이터셋
dtrain = lgb.Dataset(X_train, y_train)
dvalid = lgb.Dataset(X_valid, y_valid)

# LightGBM 모델 훈련
lgb_model = lgb.train(params=params,
                    train_set=dtrain,
                    num_boost_round=500,
                    valid_sets=(dtrain, dvalid),
                    categorical_feature=cat_features,
                    verbose_eval=50)

C:\Users\KOREAIT\.conda\envs\py39tf2\lib\site-packages\lightgbm\basic.py:2068: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['상점ID', '상품분류ID']
  _log_warning('categorical_feature in Dataset is overridden.\n'
C:\Users\KOREAIT\.conda\envs\py39tf2\lib\site-packages\lightgbm\engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Info] Total Bins 426
[LightGBM] [Info] Number of data points in the train set: 10675678, number of used features: 4
[LightGBM] [Info] Start training from score 0.299125


C:\Users\KOREAIT\.conda\envs\py39tf2\lib\site-packages\lightgbm\basic.py:1780: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overriding the parameters from Reference Dataset.')
C:\Users\KOREAIT\.conda\envs\py39tf2\lib\site-packages\lightgbm\basic.py:1513: UserWarning: categorical_column in param dict is overridden.
  _log_warning(f'{cat_alias} in param dict is overridden.')


[50]	training's rmse: 1.14777	valid_1's rmse: 1.06755
[100]	training's rmse: 1.11425	valid_1's rmse: 1.0386
[150]	training's rmse: 1.09673	valid_1's rmse: 1.02671
[200]	training's rmse: 1.08573	valid_1's rmse: 1.02027
[250]	training's rmse: 1.07722	valid_1's rmse: 1.01661
[300]	training's rmse: 1.0698	valid_1's rmse: 1.0138
[350]	training's rmse: 1.06317	valid_1's rmse: 1.01084
[400]	training's rmse: 1.05734	valid_1's rmse: 1.00936
[450]	training's rmse: 1.05224	valid_1's rmse: 1.00818
[500]	training's rmse: 1.04792	valid_1's rmse: 1.00722


- 베이스라인 모델 훈련 결과, 검증 데이터 RMSE는 1.00722
- RMSE(오차율)값은 0에 가까워야 좋음

* 범주형 데이터인 상점ID, 상품ID, 상품분류ID 중 고윳값 개수가 너무 많은 상품ID는 범주형 데이터로 취급하지 않음
(LightGBM 문서에 따르면, 고윳값 개수가 너무 많은 범주형 데이터는 수치형 데이터로 취급해야 성능이 더 잘 나옴)

# 10. 예측 및 제출 파일 생성

In [45]:
# 예측
preds = lgb_model.predict(X_test).clip(0, 20)

# 제출 파일 생성
# submission['item_cnt_month'] = preds
# submission.to_csv('submission.csv', index=False)

In [46]:
preds

array([0.58291252, 0.35683905, 0.42794278, ..., 0.11682724, 0.09855231,
       0.08657996])

In [47]:
del X_train, y_train, X_valid, y_valid, X_test, lgb_model, dtrain, dvalid
gc.collect();